In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv')
df.head()

In [ ]:
df.drop(["RowNumber","CustomerId","Surname"],axis= "columns", inplace= True)
df.shape

In [ ]:
#Converting Gender into binary
df['Gender'].replace({'Female': 0, 'Male': 1}, inplace= True)

#One hot encoding Geography column
df1 = pd.get_dummies(data= df, columns= ['Geography'])
df1.head()

In [ ]:
#Need to scale some columns
c_to_scale= ['CreditScore','Age','Balance','EstimatedSalary','Tenure']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df1[c_to_scale] = scaler.fit_transform(df1[c_to_scale])

In [ ]:
df1.dtypes

In [ ]:
#Train Test Split
X= df1.drop('Exited', axis='columns')
y= df1['Exited']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)
X_train.shape

In [ ]:
#Lets start building ANN model

import tensorflow as tf
from tensorflow import keras

In [ ]:
model= keras.Sequential([
                keras.layers.Dense(64, input_shape= (12,), activation= 'relu'),
                keras.layers.Dropout(0.4),
                keras.layers.Dense(64, activation='relu'),
                keras.layers.Dropout(0.4),
                keras.layers.Dense(1,activation='sigmoid')
])

model.compile(optimizer= 'adam',
              loss= 'binary_crossentropy',
              metrics= ['accuracy'] 
              )

In [ ]:
history= model.fit(X_train,y_train,validation_split= 0.3 ,epochs= 100,verbose= 0 )

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
yp= model.predict(X_test)

y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))
print("We see that f1 score for customer will exit prediction is very less due to highly imbalanced data ")

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred)

plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')